In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df1 = pd.read_excel("Unmatched_Data.xlsx", sheet_name="Saiba_Dump")
df2 = pd.read_excel("Unmatched_Data.xlsx", sheet_name='Lombard_Statement')

In [3]:
def preprocess_text(text):
    if pd.isna(text):
        return ""
    return str(text).lower().strip()

def find_similar_policies(df1, df2, col1, col2, threshold=61):
    # Preprocess the policy names
    df1[col1] = df1[col1].apply(preprocess_text)
    df2[col2] = df2[col2].apply(preprocess_text)

    # Get unique policy names
    unique_policies_1 = df1[col1].unique()
    unique_policies_2 = df2[col2].unique()

    # Combine the policy names
    combined_policies = pd.Series(list(unique_policies_1) + list(unique_policies_2))

    # Vectorize the policy names using TF-IDF
    vectorizer = TfidfVectorizer().fit(combined_policies)
    tfidf_matrix = vectorizer.transform(combined_policies)

    # Split the TF-IDF matrix back into the two datasets
    tfidf_matrix_1 = tfidf_matrix[:len(unique_policies_1)]
    tfidf_matrix_2 = tfidf_matrix[len(unique_policies_1):]

    # Calculate cosine similarity
    cosine_similarities = cosine_similarity(tfidf_matrix_1, tfidf_matrix_2)

    # Find the best matches
    results = []
    for i in range(len(unique_policies_1)):
        for j in range(len(unique_policies_2)):
            similarity = cosine_similarities[i, j] * 100
            if similarity >= threshold:
                results.append((similarity, unique_policies_1[i], unique_policies_2[j]))

    # Sort results by similarity percentage
    results.sort(reverse=True, key=lambda x: x[0])

    return results

In [4]:
similar_policies = find_similar_policies(df1, df2, 'Policy Type', 'PRODUCT_NAME')

for similarity, policy1, policy2 in similar_policies:
    print(f'[{similarity:.2f}%] ({policy1}) - ({policy2})')

[100.00%] (industrial all risk.) - (industrial all risk)
[100.00%] (group personal accident) - (group personal accident)
[89.10%] (bharat sookshma udyam suraksha) - (icici bharat sookshma udyam suraksha)
[89.10%] (bharat laghu udyam suraksha) - (icici bharat laghu udyam suraksha)
[83.53%] (motor private car) - (private car)
[82.45%] (bharat griha raksha) - (icici bharat griha raksha policy)
[72.94%] (marine export(open)) - (marine export import insurance(open)policy)
[70.73%] (marine import(open)) - (marine export import insurance(open)policy)
[66.17%] (standard fire and special peril policy.) - (standard fire & special perils policy)
[65.79%] (directors & officers liability) - (directors and officers policy)
[63.68%] (erection all risk) - (erection all risks)
